In [ ]:
import sys
sys.path.append("../")
import numpy as np
import csv
import os
import pandas as pd
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
theodo_u = importlib.reload(theodo_u)
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial.transform import Rotation as R_scipy

## Select date of the data to compute

In [ ]:
# Path of output
path = "../data/20230725-1/"

## 0. Import Data & Synchronize RTS Timestamp with GNSS Timestamp

In [ ]:
theodo_u = importlib.reload(theodo_u)

df_rts = pd.read_csv(path + "ground_truth/groundtruth_rts_lidar.csv", names=["Timestamp", "X", "Y", "Z", "qx", "qy", "qz", "qw"], delimiter= ' ')
df_gps = pd.read_csv(path + "ground_truth/groundtruth_gps_lidar.csv", delimiter= ',')

df_rts['Timestamp'] = pd.to_datetime(df_rts['Timestamp'], unit='s') # Convert to datetime
df_rts['Timestamp'] -= pd.Timedelta(hours=4, seconds=6) # Subtract 4 hours from the RTS Timestamp column
df_rts['Timestamp'] = df_rts["Timestamp"].values.astype(np.int64) // (1*10 ** 9) # Convert to Unix Timestamp
#df_gps['Timestamp'] = df_gps["Timestamp"].values.astype(np.float64)

## 1. Plot GPS and RTS data

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), ncols=2, nrows=1)
ax[0].scatter(df_gps['X'], df_gps['Y'], s=1, c='blue')
ax[0].set_xlabel('X [m]')
ax[0].set_ylabel('Y [m]')
ax[0].set_title('GPS raw')
ax[0].set_aspect('equal')
ax[1].scatter(df_rts['X'], df_rts['Y'], s=1, c='red')
ax[1].set_xlabel('X [m]')
ax[1].set_ylabel('Y [m]')
ax[1].set_title('RTS raw')
ax[1].set_aspect('equal')
ax[1].invert_xaxis()
ax[1].invert_yaxis()


## 2. Compute transformation matrix from RTS frame to GNSS frame : $T_{RTS}^{GNSS}$.

In [ ]:
traj_merge = pd.merge(df_rts[['Timestamp', 'X', 'Y', 'Z', 'qx','qy','qz','qw']], df_gps[['Timestamp', 'X', 'Y', 'Z', 'qx','qy','qz','qw','PDOP','Sats']], on=['Timestamp'], how='outer', indicator=True)
traj_common = traj_merge[traj_merge['_merge'] == 'both']
traj_gps_only = traj_merge[traj_merge['_merge'] == 'right_only']
traj_rts_only = traj_merge[traj_merge['_merge'] == 'left_only']

traj_common.rename(columns={'X_x':'X_rts', 'Y_x':'Y_rts', 'Z_x':'Z_rts', 'X_y':'X_gps', 'Y_y':'Y_gps', 'Z_y':'Z_gps'}, inplace=True)
traj_gps_only.rename(columns={'X_y':'X', 'Y_y':'Y', 'Z_y':'Z', 'qx_y' : 'qx', 'qy_y' : 'qy', 'qz_y' : 'qz', 'qw_y' : 'qw'}, inplace=True)
traj_rts_only.rename(columns={'X_x':'X', 'Y_x':'Y', 'Z_x':'Z', 'qx_x' : 'qx', 'qy_x' : 'qy', 'qz_x' : 'qz', 'qw_x' : 'qw'}, inplace=True)

traj_gps_only.drop(['X_x', 'Y_x', 'Z_x', 'qx_x', 'qy_x', 'qz_x', 'qw_x', '_merge'], axis=1, inplace=True)
traj_rts_only.drop(['X_y', 'Y_y', 'Z_y', 'qx_y', 'qy_y', 'qz_y', 'qw_y', 'PDOP', 'Sats', '_merge'], axis=1, inplace=True)

In [ ]:
P = np.array([traj_common['X_rts'], traj_common['Y_rts'], traj_common['Z_rts'], np.ones(len(traj_common['X_rts']))])
Q = np.array([traj_common['X_gps'], traj_common['Y_gps'], traj_common['Z_gps'], np.ones(len(traj_common['X_gps']))])

T = theodo_u.point_to_point_minimization(P, Q)
P = T @ P

traj_common['X_rts'] = P[0,:]
traj_common['Y_rts'] = P[1,:]
traj_common['Z_rts'] = P[2,:]

P_rts = np.array([df_rts['X'], df_rts['Y'], df_rts['Z'],np.ones(len(df_rts['X']))])
P_rts = T @ P_rts
df_rts['X'] = P_rts[0,:]
df_rts['Y'] = P_rts[1,:]
df_rts['Z'] = P_rts[2,:]

P_traj_rts_only = np.array([traj_rts_only['X'], traj_rts_only['Y'], traj_rts_only['Z'], np.ones(len(traj_rts_only['X']))])
P_traj_rts_only = T @ P_traj_rts_only
traj_rts_only['X'] = P_traj_rts_only[0,:]
traj_rts_only['Y'] = P_traj_rts_only[1,:]
traj_rts_only['Z'] = P_traj_rts_only[2,:]

## Check timestamp sync

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(traj_common['Timestamp'], traj_common['X_rts'], c='skyblue', label='RTS', s=10)
plt.scatter(traj_common['Timestamp'], traj_common['X_gps'], c='pink', label='GPS', s=10)
plt.legend()
plt.xlabel('Time [s]')
plt.ylabel('X [m]')

## 2. GPS only, RTS only and common trajectory

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12), ncols = 3, nrows = 1)
ax[0].scatter(df_rts['X'],df_rts['Y'], c='skyblue', label='RTS only', s=2, alpha = 0.5)
ax[1].scatter(df_gps['X'], df_gps['Y'], c='pink', label='GPS only', s=2, alpha=0.5)
ax[2].scatter(traj_common['X_rts'], traj_common['Y_rts'], c='orange', label='Common Trajectory', s=3, alpha=0.5)


ax[0].set_xlabel('X (m)')
ax[0].set_ylabel('Y (m)')
ax[0].set_title('RTS only')
ax[0].legend(loc = 'lower left')
ax[0].set_aspect('equal')

ax[1].set_xlabel('X (m)')
ax[1].set_ylabel('Y (m)')
ax[1].set_title('GPS only')
ax[1].legend(loc = 'lower left')
ax[1].set_aspect('equal')

ax[2].set_xlabel('X (m)')
ax[2].set_ylabel('Y (m)')
ax[2].set_title('RTS and GPS common trajectory')
ax[2].legend(loc = 'lower left')
ax[2].set_aspect('equal')


## 3. Final plot with common trajectory

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(traj_common['X_rts']-245610, traj_common['Y_rts']-5182375, s=1, c='lightgrey', label = 'common')
ax.scatter(traj_gps_only['X']-245610, traj_gps_only['Y']-5182375, s=3, c='lightskyblue', label = 'GPS only')
ax.scatter(traj_rts_only['X']-245610, traj_rts_only['Y']-5182375, s=1, c='lightsalmon', label = 'RTS only')
ax.set_aspect('equal')
ax.legend()
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_title('Trajectory reconstruction with RTS and GPS')
ax.set_aspect('equal')

## 5. Final comparaison between raw data and trajectory reconstruction

In [ ]:
X_origin = 245610
Y_origin = 5182375
Z_origin = 62

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15), ncols=3, nrows=1)

ax[0].scatter(df_gps['X']-X_origin, df_gps['Y']-Y_origin, s=3, c='lightskyblue', alpha=0.8)
ax[0].set_xlabel('X [m]')
ax[0].set_ylabel('Y [m]')
ax[0].set_title('GPS only')
ax[0].set_aspect('equal')

ax[1].scatter(df_rts['X']-X_origin, df_rts['Y']-Y_origin, s=1, c='lightsalmon', alpha=0.5)
ax[1].set_xlabel('X [m]')
ax[1].set_ylabel('Y [m]')
ax[1].set_title('RTS only')
ax[1].set_aspect('equal')

ax[2].scatter(traj_common['X_rts']-X_origin, traj_common['Y_rts']-Y_origin, s=1, c='lightgrey', label = 'common')
ax[2].scatter(traj_gps_only['X']-X_origin, traj_gps_only['Y']-Y_origin, s=3, c='lightskyblue', label = 'GPS only')
ax[2].scatter(traj_rts_only['X']-X_origin, traj_rts_only['Y']-Y_origin, s=1, c='lightsalmon', label = 'RTS only')
ax[2].set_aspect('equal')
ax[2].legend()
ax[2].set_xlabel('X [m]')
ax[2].set_ylabel('Y [m]')
ax[2].set_title('Trajectory reconstruction')
ax[2].set_aspect('equal')

fig.suptitle('First trajectory',fontsize=17, y=0.68)

# Figure : precision GPS and RTS

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(traj_common['X_rts'] - X_origin, traj_common['Y_rts'] - Y_origin, c='skyblue', label='RTS common', s=3, alpha=0.5)
plt.scatter(traj_common['X_gps'] - X_origin, traj_common['Y_gps'] - Y_origin, cmap = 'plasma', c=traj_common['Sats'], label='GPS common', s=3, alpha=0.3)
plt.gca().set_aspect('equal')
plt.legend(loc = 'lower left')
plt.xlabel('X [s]')
plt.ylabel('Y [m]')

colorbar = plt.colorbar(shrink=0.65)
colorbar.set_label('Number of satellites', rotation=270)
colorbar.set_ticks([min(traj_common['Sats']), max(traj_common['Sats'])])

## Interpolate the trajectory with norlab_trajectory

In [ ]:
#import norlab_trajectory_python as trajpy

## To merge images :

In [ ]:
from PIL import Image
def merge_images_vertically(image_path1, image_path2, output_path):
    # Ouvrir les deux images
    image1 = Image.open(image_path1)
    image2 = Image.open(image_path2)

    # Récupérer les dimensions des images
    width1, height1 = image1.size
    width2, height2 = image2.size

    # Calculer la largeur et la hauteur de l'image fusionnée
    merged_width = max(width1, width2)
    merged_height = height1 + height2

    # Créer une nouvelle image fusionnée
    merged_image = Image.new("RGB", (merged_width, merged_height), (255, 255, 255))

    # Coller les deux images dans l'image fusionnée
    merged_image.paste(image1, (0, 0))
    merged_image.paste(image2, (0, height1))

    # Enregistrer l'image fusionnée
    merged_image.save(output_path)

merge_images_vertically("/Users/effiedaum/Master_thesis/Figures/Plot_workshop/final_plot_1st.png", "/Users/effiedaum/Master_thesis/Figures/Plot_workshop/final_plot_2nd.png", "/Users/effiedaum/Master_thesis/Figures/Plot_workshop/final_plot_merged.png")

# 6. Error inter-prism and inter-gps

In [ ]:
theodo_u = importlib.reload(theodo_u)
prefix = "filtered_prediction/f-2-2-2-3-6-0-L"
linear_interpolation = True
if(linear_interpolation==True):
    P1 = theodo_u.read_prediction_data_Linear_csv_file(path+prefix +"_1.csv")
    P2 = theodo_u.read_prediction_data_Linear_csv_file(path+prefix +"_2.csv")
    P3 = theodo_u.read_prediction_data_Linear_csv_file(path+prefix +"_3.csv")
else:
    P1 = theodo_u.read_prediction_data_GP_csv_file(path+prefix +"_1.csv")
    P2 = theodo_u.read_prediction_data_GP_csv_file(path+prefix +"_2.csv")
    P3 = theodo_u.read_prediction_data_GP_csv_file(path+prefix +"_3.csv")
    
P1_arr = np.array(P1)
P2_arr = np.array(P2)
P3_arr = np.array(P3)

path_file_GCP = "total_stations/GCP.txt"
file_name = path+path_file_GCP
trimble_1_gcp, trimble_2_gcp, trimble_3_gcp, T_1_grand, T_2_grand, T_3_grand = theodo_u.read_marker_file(file_name, 1, 1)

P1_corrected = []
P2_corrected = []
P3_corrected = []
for i, j, k in zip(P1_arr, P2_arr, P3_arr):
    P1_arr = T_1_grand@i[1:5].T
    P1_corrected.append([i[0], P1_arr[0], P1_arr[1], P1_arr[2], 1])
    P2_arr = T_2_grand@j[1:5].T
    P2_corrected.append([j[0], P2_arr[0], P2_arr[1], P2_arr[2], 1])
    P3_arr = T_3_grand@k[1:5].T
    P3_corrected.append([k[0], P3_arr[0], P3_arr[1], P3_arr[2], 1])

P1_corrected = np.array(P1_corrected)
P2_corrected = np.array(P2_corrected)
P3_corrected = np.array(P3_corrected)

extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(path+"sensors_extrinsic_calibration/calibration_results.csv")

In [ ]:
print(extrinsic_calibration_results)

In [ ]:
d12_rts = []
d13_rts = []
d23_rts = []
diff_time_synchro = 4*3600+6
for i,j,k in zip(P1_corrected,P2_corrected,P3_corrected):
    d12_rts.append([i[0]-diff_time_synchro,1000*abs(np.linalg.norm(i[1:4]-j[1:4])-extrinsic_calibration_results[0])])
    d13_rts.append([i[0]-diff_time_synchro,1000*abs(np.linalg.norm(i[1:4]-k[1:4])-extrinsic_calibration_results[1])])
    d23_rts.append([j[0]-diff_time_synchro,1000*abs(np.linalg.norm(k[1:4]-j[1:4])-extrinsic_calibration_results[2])])

In [ ]:
xs_rts = [None] * len(traj_common['Timestamp'])
for i in range(0,len(d12_rts)):
    out = next(iter(np.where(traj_common['Timestamp'] == np.int64(d12_rts[i][0]))), 'Nan')
    for j in out:
        if j!='Nan':
            if np.mean([d12_rts[i][1],d13_rts[i][1],d23_rts[i][1]])<10000:
                xs_rts[j]= np.mean([d12_rts[i][1],d13_rts[i][1],d23_rts[i][1]])
traj_common['Inter_prism'] = xs_rts

In [ ]:
import matplotlib.colors as colors
plt.figure(figsize=(10,8))
plt.scatter(traj_common['X_rts'] - X_origin, traj_common['Y_rts'] - Y_origin, cmap = 'plasma', norm=colors.LogNorm(vmin=0.1, vmax=100), c=traj_common['Inter_prism'], label='Rts common', s=3, alpha=0.3)
#plt.scatter(traj_common['X_gps'] - X_origin, traj_common['Y_gps'] - Y_origin, cmap = 'plasma', c=traj_common['Sats'], label='GPS common', s=3, alpha=0.3)
plt.gca().set_aspect('equal')
plt.legend(loc = 'lower left')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')

colorbar = plt.colorbar(shrink=0.65)
colorbar.set_label('Inter-prism error', rotation=270)
# colorbar.set_ticks([min(traj_common['Inter-prism']), max(traj_common['Inter-prism'])])

In [ ]:
df_gps_1 = pd.read_csv(path + "gps_data/rover_1_raw.csv", names=['Point ID','Easting','Northing','Elevation','H. Precision','V. Precision','PDOP','Sats','Datetime','Timestamp'], delimiter= ',', skiprows=[0])
df_gps_2 = pd.read_csv(path + "gps_data/rover_2_raw.csv", names=['Point ID','Easting','Northing','Elevation','H. Precision','V. Precision','PDOP','Sats','Datetime','Timestamp'], delimiter= ',', skiprows=[0])
df_gps_3 = pd.read_csv(path + "gps_data/rover_3_raw.csv", names=['Point ID','Easting','Northing','Elevation','H. Precision','V. Precision','PDOP','Sats','Datetime','Timestamp'], delimiter= ',', skiprows=[0])

In [ ]:
xs_gps = [None] * len(traj_common['Timestamp'])
print(len(traj_common['Timestamp']))
for i in range(0,len(traj_common['Timestamp'])):
    try:
        if traj_common['Timestamp'][i] != 'None' :
            out1 = next(iter(np.where(df_gps_1['Timestamp'] == traj_common['Timestamp'][i])[0]), 'Nan')
            out2 = next(iter(np.where(df_gps_2['Timestamp'] == traj_common['Timestamp'][i])[0]), 'Nan')
            out3 = next(iter(np.where(df_gps_3['Timestamp'] == traj_common['Timestamp'][i])[0]), 'Nan')
            if out1!='Nan' and out2!='Nan' and out3!='Nan':
                point_1 = np.array([np.float(df_gps_1['Easting'][out1]),np.float(df_gps_1['Northing'][out1]),np.float(df_gps_1['Elevation'][out1])])
                point_2 = np.array([np.float(df_gps_2['Easting'][out2]),np.float(df_gps_2['Northing'][out2]),np.float(df_gps_2['Elevation'][out2])])
                point_3 = np.array([np.float(df_gps_3['Easting'][out3]),np.float(df_gps_3['Northing'][out3]),np.float(df_gps_3['Elevation'][out3])])
                d12_gps = 1000*abs(np.linalg.norm(point_1-point_2)-extrinsic_calibration_results[3])
                d13_gps = 1000*abs(np.linalg.norm(point_1-point_3)-extrinsic_calibration_results[4])
                d23_gps = 1000*abs(np.linalg.norm(point_2-point_3)-extrinsic_calibration_results[5])
                if np.mean([d12_gps,d13_gps,d23_gps])<10000:
                    xs_gps[i]= np.mean([d12_gps,d13_gps,d23_gps])
    except:
        print('error')
traj_common['Inter_gps'] = xs_gps

In [ ]:
import matplotlib.colors as colors
plt.figure(figsize=(10,8))
plt.scatter(traj_common['X_gps'] - X_origin, traj_common['Y_gps'] - Y_origin, cmap = 'plasma', norm=colors.LogNorm(vmin=0.1, vmax=100), c=traj_common['Inter_gps'], label='Gps common', s=3, alpha=0.3)
#plt.scatter(traj_common['X_gps'] - X_origin, traj_common['Y_gps'] - Y_origin, cmap = 'plasma', c=traj_common['Sats'], label='GPS common', s=3, alpha=0.3)
plt.gca().set_aspect('equal')
plt.legend(loc = 'lower left')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')

colorbar = plt.colorbar(shrink=0.65)
colorbar.set_label('Inter-gps error', rotation=270)
#colorbar.set_ticks([min(traj_common['Inter-gps']), max(traj_common['Inter-gps'])])

# 7. Save error inter-prism and inter-gps in vtk

In [ ]:
#import required packages
import itertools
import numpy as np
import pyvista as pv
import math

pointSec = []
inter_gps = []
for index, valuesx in traj_common.iterrows():
    if traj_common.loc[index].Inter_gps!='None' and math.isnan(traj_common.loc[index].Inter_gps)!=True:
        x, y, z = traj_common.loc[index].X_gps, traj_common.loc[index].Y_gps, traj_common.loc[index].Z_gps
        inter_gps.append(traj_common.loc[index].Inter_gps)
        pointSec.append([x- X_origin,y- Y_origin,z- Z_origin])
    
pointSec = np.array(pointSec)
inter_gps = np.array(inter_gps)
point_cloud = pv.PolyData(pointSec)
data = points[:,-1]
point_cloud["Inter_gps"]=inter_gps
point_cloud.plot(render_points_as_spheres=True)
point_cloud.save('/home/maxime/gps_traj_common.vtk',binary=False)

pointSec = []
inter_rts = []
for index, valuesx in traj_common.iterrows():
    if traj_common.loc[index].Inter_prism!='None' and math.isnan(traj_common.loc[index].Inter_prism)!=True:
        x, y, z = traj_common.loc[index].X_rts, traj_common.loc[index].Y_rts, traj_common.loc[index].Z_rts
        inter_rts.append(traj_common.loc[index].Inter_prism)
        pointSec.append([x- X_origin,y- Y_origin,z- Z_origin])
    
pointSec = np.array(pointSec)
inter_rts = np.array(inter_rts)
point_cloud = pv.PolyData(pointSec)
data = points[:,-1]
point_cloud["Inter_prism"]=inter_rts
point_cloud.plot(render_points_as_spheres=True)
point_cloud.save('/home/maxime/rts_traj_common.vtk',binary=False)